# Forecast ML Baseline - Предсказание отклонения BTC от тренда

## Формулировка задачи в терминах ML

**Задача:** Предсказать отклонение BTC от тренда на 1ч/4ч/24ч

**Тип задачи:** 
- **Регрессия**: предсказать величину отклонения (residual-таргет, лог-отклонение от MA_L)
- **Классификация**: предсказать направление движения (P(up))

**Целевые переменные (согласно плану):**
- `y_H(t) = log(P_{t+H}) - log(MA_L(t))` - логарифмическое отклонение от тренда
- `d_H(t) = 1[y_H(t) > 0]` - направление (1 = вверх, 0 = вниз)
- где `MA_L` - EMA/KAMA, рассчитанная строго до момента `t` (без утечек)

**Признаки (ядро MVP, ~10 согласно плану):**
1. $z$-score цены к $\mathrm{MA}_L$
2. Realized vol $\sigma_{20}$ (на целевом ТФ)
3. ATR% (14)
4. RSI(14) (+ бинарный «RSI > 50»)
5. Нормированный моментум (MACD-hist/ATR)
6. Funding z-score; Basis z-score (если доступны)
7. $\Delta\mathrm{OI}\%$, нормированная на объём (если доступны)
8. Интенсивность ликвидаций (если доступны)
9. Для 24ч — DXY $\Delta 1d$ (если доступны)
10. Исторические отклонения от MA (лаги 1–6)

**Продуктовая метрика:** 
- MAE-gain ≥ 5%/6%/8% (1ч/4ч/24ч) vs нулевой прогноз
- Sharpe ≥ 1.0/1.1/1.2 для policy long-if-ŷ>0

**ML-метрики:**
- **Регрессия**: MAE, RMSE, R²
- **Классификация**: Accuracy, Precision, Recall, F1-score, ROC-AUC
- **Калибровка**: Brier Score, ECE (Expected Calibration Error)


## 1. Загрузка и очистка данных


In [ ]:
import sys
from pathlib import Path

# Добавляем путь к проекту для импорта модулей
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import sqlite3
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Импортируем модули проекта
try:
    from app.ml.data_adapter import load_bars_from_project, make_loader
    from app.ml.features import build_features
    from app.infrastructure.db import DB
    USE_PROJECT_MODULES = True
except ImportError:
    print("⚠️ Не удалось импортировать модули проекта, используем упрощённую версию")
    USE_PROJECT_MODULES = False

# Параметры
SYMBOL = "BTC"
TIMEFRAME = "1h"  # Можно изменить на "4h" или "24h"
HORIZON = 24  # Горизонт прогноза в барах
DB_PATH = '../data/alt_forecast.db'  # Путь к базе данных

print(f"📊 Параметры:")
print(f"   Symbol: {SYMBOL}")
print(f"   Timeframe: {TIMEFRAME}")
print(f"   Horizon: {HORIZON} баров")


In [ ]:
def load_data_simple(db_path: str, symbol: str, timeframe: str, limit: int = 5000) -> pd.DataFrame:
    """Упрощённая загрузка данных из SQLite."""
    conn = sqlite3.connect(db_path)
    
    # Пробуем найти таблицу с барами
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cur.fetchall()]
    
    # Ищем таблицу bars
    table_name = None
    for table in tables:
        cur.execute(f"PRAGMA table_info({table})")
        cols = [row[1].lower() for row in cur.fetchall()]
        if {'metric', 'timeframe', 'ts', 'o', 'h', 'l', 'c'}.issubset(set(cols)):
            table_name = table
            break
    
    if not table_name:
        raise ValueError(f"Не найдена таблица с барами в {db_path}")
    
    # Загружаем данные
    query = f"""
        SELECT ts, o as open, h as high, l as low, c as close, v as volume
        FROM {table_name}
        WHERE metric = ? AND timeframe = ?
        ORDER BY ts DESC
        LIMIT ?
    """
    
    df = pd.read_sql_query(query, conn, params=[symbol, timeframe, limit])
    conn.close()
    
    if df.empty:
        raise ValueError(f"Нет данных для {symbol} {timeframe}")
    
    # Нормализуем
    df['ts'] = pd.to_datetime(df['ts'], unit='ms', utc=True)
    df = df.sort_values('ts').reset_index(drop=True)
    
    return df

# Загружаем данные
if USE_PROJECT_MODULES:
    try:
        # Пробуем использовать DB из проекта
        db = DB(DB_PATH)
        loader = make_loader(db)
        df = loader(SYMBOL, TIMEFRAME, limit=5000)
        print(f"✅ Загружено {len(df)} баров из проекта")
    except Exception as e:
        print(f"⚠️ Ошибка загрузки через проект: {e}")
        print("📊 Пробуем упрощённую загрузку...")
        if Path(DB_PATH).exists():
            df = load_data_simple(DB_PATH, SYMBOL, TIMEFRAME, limit=5000)
            print(f"✅ Загружено {len(df)} баров (упрощённый метод)")
        else:
            raise FileNotFoundError(f"База данных не найдена: {DB_PATH}")
else:
    if Path(DB_PATH).exists():
        df = load_data_simple(DB_PATH, SYMBOL, TIMEFRAME, limit=5000)
        print(f"✅ Загружено {len(df)} баров")
    else:
        raise FileNotFoundError(f"База данных не найдена: {DB_PATH}")

print(f"\n📊 Размер данных: {len(df)} строк")
print(f"📊 Период: {df['ts'].min()} - {df['ts'].max()}")
print(f"📊 Колонки: {list(df.columns)}")


In [ ]:
# Очистка данных
print(f"📊 Исходный размер: {len(df)} строк")
print(f"\nПропущенные значения:")
print(df.isnull().sum())

# Удаляем строки с пропусками в критических колонках
critical_cols = ['open', 'high', 'low', 'close']
df_clean = df.dropna(subset=critical_cols).copy()

# Заполняем пропуски в volume нулями
if 'volume' in df_clean.columns:
    df_clean['volume'].fillna(0, inplace=True)
else:
    df_clean['volume'] = 0.0

# Проверяем корректность данных
df_clean = df_clean[df_clean['high'] >= df_clean['low']]
df_clean = df_clean[df_clean['open'] > 0]
df_clean = df_clean[df_clean['close'] > 0]

# Удаляем дубликаты по времени
df_clean = df_clean.drop_duplicates(subset=['ts'], keep='last')
df_clean = df_clean.sort_values('ts').reset_index(drop=True)

print(f"\n✅ После очистки: {len(df_clean)} строк")
print(f"Удалено строк: {len(df) - len(df_clean)}")

df = df_clean


## 2. Построение признаков и целевых переменных


In [ ]:
# Строим признаки
if USE_PROJECT_MODULES:
    try:
        df_feat = build_features(df)
        print("✅ Признаки построены через build_features()")
    except Exception as e:
        print(f"⚠️ Ошибка build_features: {e}")
        USE_PROJECT_MODULES = False

if not USE_PROJECT_MODULES:
    # Упрощённая версия построения признаков
    df_feat = df.copy()
    df_feat = df_feat.sort_values('ts').reset_index(drop=True)
    df_feat.set_index('ts', inplace=True)
    
    # Returns
    df_feat['ret_1'] = df_feat['close'].pct_change(1).fillna(0.0)
    df_feat['ret_3'] = df_feat['close'].pct_change(3).fillna(0.0)
    df_feat['ret_6'] = df_feat['close'].pct_change(6).fillna(0.0)
    df_feat['ret_24'] = df_feat['close'].pct_change(24).fillna(0.0)
    
    # Volatility
    df_feat['vol_24'] = df_feat['ret_1'].rolling(24).std().fillna(0.0)
    df_feat['vol_72'] = df_feat['ret_1'].rolling(72).std().fillna(0.0)
    
    # RSI (упрощённый)
    delta = df_feat['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-9)
    df_feat['rsi_14'] = 100 - (100 / (1 + rs)).fillna(50.0)
    
    # MACD
    ema12 = df_feat['close'].ewm(span=12, adjust=False).mean()
    ema26 = df_feat['close'].ewm(span=26, adjust=False).mean()
    df_feat['macd'] = (ema12 - ema26).fillna(0.0)
    df_feat['macd_sig'] = df_feat['macd'].ewm(span=9, adjust=False).mean().fillna(0.0)
    
    # Bollinger Bands width
    ma20 = df_feat['close'].rolling(20).mean()
    std20 = df_feat['close'].rolling(20).std()
    df_feat['bb_width'] = (2 * std20 / (ma20 + 1e-9)).fillna(0.0)
    
    # Volume relative
    df_feat['vol_rel_24'] = df_feat['volume'] / (df_feat['volume'].rolling(24).mean() + 1e-9).fillna(1.0)
    
    # Очистка
    df_feat = df_feat.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    print("✅ Признаки построены (упрощённый метод)")

print(f"\n📊 Признаки: {[col for col in df_feat.columns if col not in ['open', 'high', 'low', 'close', 'volume']]}")
print(f"📊 Размер данных с признаками: {len(df_feat)} строк")


In [ ]:
# Строим целевые переменные
# Регрессия: forward return (процентное изменение через horizon баров)
y_reg = df_feat['close'].pct_change(HORIZON).shift(-HORIZON)

# Классификация: направление (1 = вверх, 0 = вниз)
y_cls = (y_reg > 0).astype(int)

# Убираем последние HORIZON строк (у них нет целевой переменной)
df_feat = df_feat.iloc[:-HORIZON].copy()
y_reg = y_reg.iloc[:-HORIZON]
y_cls = y_cls.iloc[:-HORIZON]

# Убираем OHLCV из признаков
feature_cols = [col for col in df_feat.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
X = df_feat[feature_cols].copy()

# Удаляем строки с NaN в целевых переменных
valid_idx = ~(y_reg.isna() | y_cls.isna())
X = X[valid_idx].copy()
y_reg = y_reg[valid_idx].copy()
y_cls = y_cls[valid_idx].copy()

print(f"✅ Подготовлено данных:")
print(f"   Признаков: {len(feature_cols)}")
print(f"   Образцов: {len(X)}")
print(f"   Распределение классов: {y_cls.value_counts().to_dict()}")
print(f"   Средний return: {y_reg.mean():.4f} ({y_reg.mean()*100:.2f}%)")
print(f"   Std return: {y_reg.std():.4f} ({y_reg.std()*100:.2f}%)")


## 3. Анализ данных


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Распределение целевой переменной (регрессия)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

y_reg.hist(bins=50, ax=axes[0], color='steelblue', alpha=0.7)
axes[0].set_title(f'Распределение forward return (horizon={HORIZON})')
axes[0].set_xlabel('Return')
axes[0].set_ylabel('Частота')
axes[0].axvline(y_reg.mean(), color='red', linestyle='--', label=f'Mean: {y_reg.mean():.4f}')
axes[0].legend()

# Распределение целевой переменной (классификация)
y_cls.value_counts().plot(kind='bar', ax=axes[1], color=['red', 'green'])
axes[1].set_title('Распределение направления (классификация)')
axes[1].set_xlabel('Direction (0=down, 1=up)')
axes[1].set_ylabel('Количество')
axes[1].set_xticklabels(['Down', 'Up'], rotation=0)

plt.tight_layout()
plt.show()

print(f"📊 Статистика forward return:")
print(f"   Mean: {y_reg.mean():.6f}")
print(f"   Std: {y_reg.std():.6f}")
print(f"   Min: {y_reg.min():.6f}")
print(f"   Max: {y_reg.max():.6f}")
print(f"   Median: {y_reg.median():.6f}")
print(f"\n📊 Распределение классов:")
print(f"   Up (1): {y_cls.sum()} ({y_cls.mean()*100:.2f}%)")
print(f"   Down (0): {(1-y_cls).sum()} ({(1-y_cls.mean())*100:.2f}%)")


In [ ]:
# Корреляция признаков с целевой переменной
correlations = X.corrwith(y_reg).sort_values(ascending=False)
print("📊 Корреляция признаков с forward return:")
print(correlations)

# Визуализация
plt.figure(figsize=(10, 6))
correlations.plot(kind='barh', color='steelblue')
plt.title('Корреляция признаков с целевой переменной (forward return)')
plt.xlabel('Корреляция')
plt.tight_layout()
plt.show()


## 4. Подготовка данных для моделирования


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Разделяем на train/test (80/20)
# Используем временное разделение (не shuffle) для временных рядов
split_idx = int(0.8 * len(X))
X_train = X.iloc[:split_idx].copy()
X_test = X.iloc[split_idx:].copy()
y_reg_train = y_reg.iloc[:split_idx].copy()
y_reg_test = y_reg.iloc[split_idx:].copy()
y_cls_train = y_cls.iloc[:split_idx].copy()
y_cls_test = y_cls.iloc[split_idx:].copy()

# Масштабируем признаки
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Разделение данных:")
print(f"   Train: {len(X_train)} образцов ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test: {len(X_test)} образцов ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n📊 Train статистика:")
print(f"   Mean return: {y_reg_train.mean():.6f}")
print(f"   Up rate: {y_cls_train.mean():.2%}")
print(f"\n📊 Test статистика:")
print(f"   Mean return: {y_reg_test.mean():.6f}")
print(f"   Up rate: {y_cls_test.mean():.2%}")


## 5. Модели

### 5.1. Простая модель 1: Linear Regression + Logistic Regression


In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

# Регрессия
lr_reg = LinearRegression()
lr_reg.fit(X_train_scaled, y_reg_train)
y_reg_train_pred = lr_reg.predict(X_train_scaled)
y_reg_test_pred = lr_reg.predict(X_test_scaled)

# Классификация
lr_cls = LogisticRegression(random_state=42, max_iter=1000)
lr_cls.fit(X_train_scaled, y_cls_train)
y_cls_train_pred = lr_cls.predict(X_train_scaled)
y_cls_test_pred = lr_cls.predict(X_test_scaled)
y_cls_test_proba = lr_cls.predict_proba(X_test_scaled)[:, 1]

# Метрики регрессии
lr_reg_metrics = {
    'model': 'Linear Regression',
    'train_mae': mean_absolute_error(y_reg_train, y_reg_train_pred),
    'test_mae': mean_absolute_error(y_reg_test, y_reg_test_pred),
    'test_rmse': np.sqrt(mean_squared_error(y_reg_test, y_reg_test_pred)),
    'test_r2': r2_score(y_reg_test, y_reg_test_pred),
}

# Метрики классификации
lr_cls_metrics = {
    'model': 'Logistic Regression',
    'train_accuracy': accuracy_score(y_cls_train, y_cls_train_pred),
    'test_accuracy': accuracy_score(y_cls_test, y_cls_test_pred),
    'test_precision': precision_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_recall': recall_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_f1': f1_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_roc_auc': roc_auc_score(y_cls_test, y_cls_test_proba) if len(np.unique(y_cls_test)) > 1 else 0.5
}

print("📊 Linear Regression - Метрики регрессии:")
for key, value in lr_reg_metrics.items():
    if key != 'model':
        print(f"  {key}: {value:.6f}")

print("\n📊 Logistic Regression - Метрики классификации:")
for key, value in lr_cls_metrics.items():
    if key != 'model':
        print(f"  {key}: {value:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Регрессия
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
rf_reg.fit(X_train_scaled, y_reg_train)
y_reg_train_pred = rf_reg.predict(X_train_scaled)
y_reg_test_pred = rf_reg.predict(X_test_scaled)

# Классификация
rf_cls = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
rf_cls.fit(X_train_scaled, y_cls_train)
y_cls_train_pred = rf_cls.predict(X_train_scaled)
y_cls_test_pred = rf_cls.predict(X_test_scaled)
y_cls_test_proba = rf_cls.predict_proba(X_test_scaled)[:, 1]

# Метрики регрессии
rf_reg_metrics = {
    'model': 'Random Forest Regressor',
    'train_mae': mean_absolute_error(y_reg_train, y_reg_train_pred),
    'test_mae': mean_absolute_error(y_reg_test, y_reg_test_pred),
    'test_rmse': np.sqrt(mean_squared_error(y_reg_test, y_reg_test_pred)),
    'test_r2': r2_score(y_reg_test, y_reg_test_pred),
}

# Метрики классификации
rf_cls_metrics = {
    'model': 'Random Forest Classifier',
    'train_accuracy': accuracy_score(y_cls_train, y_cls_train_pred),
    'test_accuracy': accuracy_score(y_cls_test, y_cls_test_pred),
    'test_precision': precision_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_recall': recall_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_f1': f1_score(y_cls_test, y_cls_test_pred, zero_division=0),
    'test_roc_auc': roc_auc_score(y_cls_test, y_cls_test_proba) if len(np.unique(y_cls_test)) > 1 else 0.5
}

print("📊 Random Forest Regressor - Метрики регрессии:")
for key, value in rf_reg_metrics.items():
    if key != 'model':
        print(f"  {key}: {value:.6f}")

print("\n📊 Random Forest Classifier - Метрики классификации:")
for key, value in rf_cls_metrics.items():
    if key != 'model':
        print(f"  {key}: {value:.4f}")

# Важность признаков
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_reg.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n📊 Топ-10 важных признаков (Random Forest):")
print(feature_importance.head(10))

# Визуализация
plt.figure(figsize=(10, 6))
feature_importance.head(10).plot(x='feature', y='importance', kind='barh', color='steelblue')
plt.title('Важность признаков (Random Forest Regressor)')
plt.xlabel('Важность')
plt.tight_layout()
plt.show()


### 5.3. Усложнённая модель: XGBoost с оптимизацией гиперпараметров


In [ ]:
try:
    import xgboost as xgb
    from sklearn.model_selection import GridSearchCV
    
    # Регрессия
    print("🔍 Оптимизация гиперпараметров XGBoost Regressor...")
    xgb_reg_base = xgb.XGBRegressor(random_state=42, eval_metric='rmse')
    
    param_grid_reg = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.1, 0.3],
    }
    
    grid_search_reg = GridSearchCV(
        xgb_reg_base, param_grid_reg, cv=3, scoring='neg_mean_absolute_error', 
        n_jobs=-1, verbose=0
    )
    
    grid_search_reg.fit(X_train_scaled, y_reg_train)
    xgb_reg = grid_search_reg.best_estimator_
    
    y_reg_train_pred = xgb_reg.predict(X_train_scaled)
    y_reg_test_pred = xgb_reg.predict(X_test_scaled)
    
    # Классификация
    print("🔍 Оптимизация гиперпараметров XGBoost Classifier...")
    xgb_cls_base = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    
    param_grid_cls = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.1, 0.3],
    }
    
    grid_search_cls = GridSearchCV(
        xgb_cls_base, param_grid_cls, cv=3, scoring='roc_auc', 
        n_jobs=-1, verbose=0
    )
    
    grid_search_cls.fit(X_train_scaled, y_cls_train)
    xgb_cls = grid_search_cls.best_estimator_
    
    y_cls_train_pred = xgb_cls.predict(X_train_scaled)
    y_cls_test_pred = xgb_cls.predict(X_test_scaled)
    y_cls_test_proba = xgb_cls.predict_proba(X_test_scaled)[:, 1]
    
    # Метрики регрессии
    xgb_reg_metrics = {
        'model': 'XGBoost Regressor (Optimized)',
        'train_mae': mean_absolute_error(y_reg_train, y_reg_train_pred),
        'test_mae': mean_absolute_error(y_reg_test, y_reg_test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_reg_test, y_reg_test_pred)),
        'test_r2': r2_score(y_reg_test, y_reg_test_pred),
    }
    
    # Метрики классификации
    xgb_cls_metrics = {
        'model': 'XGBoost Classifier (Optimized)',
        'train_accuracy': accuracy_score(y_cls_train, y_cls_train_pred),
        'test_accuracy': accuracy_score(y_cls_test, y_cls_test_pred),
        'test_precision': precision_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_recall': recall_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_f1': f1_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_roc_auc': roc_auc_score(y_cls_test, y_cls_test_proba) if len(np.unique(y_cls_test)) > 1 else 0.5
    }
    
    print(f"\n✅ Лучшие параметры Regressor: {grid_search_reg.best_params_}")
    print(f"✅ Лучшие параметры Classifier: {grid_search_cls.best_params_}")
    
    print("\n📊 XGBoost Regressor - Метрики регрессии:")
    for key, value in xgb_reg_metrics.items():
        if key != 'model':
            print(f"  {key}: {value:.6f}")
    
    print("\n📊 XGBoost Classifier - Метрики классификации:")
    for key, value in xgb_cls_metrics.items():
        if key != 'model':
            print(f"  {key}: {value:.4f}")
    
except ImportError:
    print("⚠️ XGBoost не установлен. Установите: pip install xgboost")
    print("📊 Используем Gradient Boosting...")
    
    from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
    
    gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=5)
    gb_reg.fit(X_train_scaled, y_reg_train)
    y_reg_test_pred = gb_reg.predict(X_test_scaled)
    
    gb_cls = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5)
    gb_cls.fit(X_train_scaled, y_cls_train)
    y_cls_test_pred = gb_cls.predict(X_test_scaled)
    y_cls_test_proba = gb_cls.predict_proba(X_test_scaled)[:, 1]
    
    xgb_reg_metrics = {
        'model': 'Gradient Boosting Regressor',
        'test_mae': mean_absolute_error(y_reg_test, y_reg_test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_reg_test, y_reg_test_pred)),
        'test_r2': r2_score(y_reg_test, y_reg_test_pred),
    }
    
    xgb_cls_metrics = {
        'model': 'Gradient Boosting Classifier',
        'test_accuracy': accuracy_score(y_cls_test, y_cls_test_pred),
        'test_precision': precision_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_recall': recall_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_f1': f1_score(y_cls_test, y_cls_test_pred, zero_division=0),
        'test_roc_auc': roc_auc_score(y_cls_test, y_cls_test_proba) if len(np.unique(y_cls_test)) > 1 else 0.5
    }


In [ ]:
# Собираем все метрики
all_reg_metrics = [lr_reg_metrics, rf_reg_metrics, xgb_reg_metrics]
all_cls_metrics = [lr_cls_metrics, rf_cls_metrics, xgb_cls_metrics]

reg_metrics_df = pd.DataFrame(all_reg_metrics)
cls_metrics_df = pd.DataFrame(all_cls_metrics)

print("📊 Сравнение моделей - Регрессия:")
print(reg_metrics_df.to_string(index=False))

print("\n📊 Сравнение моделей - Классификация:")
print(cls_metrics_df.to_string(index=False))


In [ ]:
# Визуализация метрик регрессии
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

reg_metric_cols = ['test_mae', 'test_rmse', 'test_r2', 'train_mae']
reg_metric_names = ['Test MAE', 'Test RMSE', 'Test R²', 'Train MAE']

for i, (col, name) in enumerate(zip(reg_metric_cols, reg_metric_names)):
    if col in reg_metrics_df.columns:
        reg_metrics_df.plot(x='model', y=col, kind='bar', ax=axes[i], color='steelblue')
        axes[i].set_title(f'{name}')
        axes[i].set_ylabel(name)
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].legend().remove()

plt.tight_layout()
plt.show()

# Визуализация метрик классификации
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

cls_metric_cols = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1', 'test_roc_auc', 'train_accuracy']
cls_metric_names = ['Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1', 'Test ROC-AUC', 'Train Accuracy']

for i, (col, name) in enumerate(zip(cls_metric_cols, cls_metric_names)):
    if col in cls_metrics_df.columns:
        cls_metrics_df.plot(x='model', y=col, kind='bar', ax=axes[i], color='steelblue')
        axes[i].set_title(f'{name}')
        axes[i].set_ylabel(name)
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].legend().remove()

plt.tight_layout()
plt.show()


In [ ]:
# Совокупная метрика качества
# Для регрессии: комбинация MAE и R²
def calculate_reg_composite_score(row):
    mae = row.get('test_mae', 1.0)
    r2 = row.get('test_r2', 0.0)
    # Нормализуем MAE (меньше = лучше) и R² (больше = лучше)
    # Используем: -MAE (нормализованный) + R²
    # MAE обычно в диапазоне [0, 0.1] для returns, нормализуем к [0, 1]
    mae_norm = max(0, 1 - mae * 10)  # Если MAE < 0.1, то mae_norm > 0
    return 0.5 * mae_norm + 0.5 * max(0, r2)

# Для классификации: комбинация F1 и ROC-AUC
def calculate_cls_composite_score(row):
    f1 = row.get('test_f1', 0.0)
    roc_auc = row.get('test_roc_auc', 0.5)
    return 0.5 * f1 + 0.5 * roc_auc

reg_metrics_df['composite_score'] = reg_metrics_df.apply(calculate_reg_composite_score, axis=1)
cls_metrics_df['composite_score'] = cls_metrics_df.apply(calculate_cls_composite_score, axis=1)

print("📊 Совокупная метрика качества - Регрессия:")
print(reg_metrics_df[['model', 'composite_score']].sort_values('composite_score', ascending=False))

print("\n📊 Совокупная метрика качества - Классификация:")
print(cls_metrics_df[['model', 'composite_score']].sort_values('composite_score', ascending=False))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

reg_metrics_df.sort_values('composite_score', ascending=True).plot(
    x='model', y='composite_score', kind='barh', ax=axes[0], color='steelblue'
)
axes[0].set_title('Совокупная метрика качества - Регрессия')
axes[0].set_xlabel('Composite Score')

cls_metrics_df.sort_values('composite_score', ascending=True).plot(
    x='model', y='composite_score', kind='barh', ax=axes[1], color='steelblue'
)
axes[1].set_title('Совокупная метрика качества - Классификация')
axes[1].set_xlabel('Composite Score')

plt.tight_layout()
plt.show()


In [ ]:
# Продуктовые метрики: MAE-gain и Sharpe
# MAE-gain = улучшение MAE по сравнению с baseline (нулевой прогноз)
baseline_mae = np.abs(y_reg_test).mean()  # MAE нулевого прогноза (всегда предсказываем 0)

print("=" * 80)
print("📊 ПРОДУКТОВЫЕ МЕТРИКИ")
print("=" * 80)

for _, row in reg_metrics_df.iterrows():
    model_name = row['model']
    test_mae = row['test_mae']
    mae_gain = (baseline_mae - test_mae) / baseline_mae * 100  # Процент улучшения
    print(f"\n{model_name}:")
    print(f"  Baseline MAE (нулевой прогноз): {baseline_mae:.6f}")
    print(f"  Test MAE: {test_mae:.6f}")
    print(f"  MAE-gain: {mae_gain:.2f}%")

# Sharpe ratio для стратегии long-if-ŷ>0
print("\n📊 Sharpe Ratio для стратегии 'long-if-ŷ>0':")
models_reg = [
    ('Linear Regression', lr_reg),
    ('Random Forest', rf_reg),
]

if 'xgb_reg' in locals():
    models_reg.append(('XGBoost', xgb_reg))
elif 'gb_reg' in locals():
    models_reg.append(('Gradient Boosting', gb_reg))

for model_name, model_reg in models_reg:
    y_pred = model_reg.predict(X_test_scaled)
    # Стратегия: покупаем если предсказанный return > 0
    positions = (y_pred > 0).astype(int)
    # Реальные returns для тех баров, где мы в позиции
    strategy_returns = y_reg_test * positions
    if strategy_returns.std() > 0:
        sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # Годовая
    else:
        sharpe = 0.0
    print(f"  {model_name}: {sharpe:.4f}")

print("\n" + "=" * 80)


In [ ]:
# ROC-кривые для классификаторов
from sklearn.metrics import roc_curve

models_cls = {
    'Logistic Regression': (lr_cls, X_test_scaled),
    'Random Forest': (rf_cls, X_test_scaled),
}

if 'xgb_cls' in locals():
    models_cls['XGBoost'] = (xgb_cls, X_test_scaled)
elif 'gb_cls' in locals():
    models_cls['Gradient Boosting'] = (gb_cls, X_test_scaled)

plt.figure(figsize=(10, 8))

for name, (model, X_data) in models_cls.items():
    y_proba = model.predict_proba(X_data)[:, 1]
    fpr, tpr, _ = roc_curve(y_cls_test, y_proba)
    auc_score = roc_auc_score(y_cls_test, y_proba) if len(np.unique(y_cls_test)) > 1 else 0.5
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.500)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривые моделей классификации')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 7. Анализ результатов и выводы


In [ ]:
print("=" * 80)
print("📊 ИТОГОВЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 80)

# Лучшая модель по совокупной метрике
best_reg_row = reg_metrics_df.loc[reg_metrics_df['composite_score'].idxmax()]
best_cls_row = cls_metrics_df.loc[cls_metrics_df['composite_score'].idxmax()]

print(f"\n🏆 Лучшая модель регрессии: {best_reg_row['model']}")
print(f"   Composite Score: {best_reg_row['composite_score']:.4f}")
print(f"   Test MAE: {best_reg_row['test_mae']:.6f}")
print(f"   Test R²: {best_reg_row['test_r2']:.4f}")

print(f"\n🏆 Лучшая модель классификации: {best_cls_row['model']}")
print(f"   Composite Score: {best_cls_row['composite_score']:.4f}")
print(f"   Test Accuracy: {best_cls_row['test_accuracy']:.4f}")
print(f"   Test F1-Score: {best_cls_row['test_f1']:.4f}")
print(f"   Test ROC-AUC: {best_cls_row['test_roc_auc']:.4f}")

# Сравнение с baseline
print(f"\n📊 Сравнение с baseline:")
print(f"   Baseline MAE (нулевой прогноз): {baseline_mae:.6f}")
print(f"   Улучшение MAE: {(baseline_mae - best_reg_row['test_mae']) / baseline_mae * 100:.2f}%")

# Проверка бенчмарков из lesson_03
target_mae_gain = {1: 5, 4: 6, 24: 8}.get(HORIZON, 5)  # Процент в зависимости от горизонта
target_auc = {1: 0.58, 4: 0.60, 24: 0.62}.get(HORIZON, 0.58)

mae_gain_actual = (baseline_mae - best_reg_row['test_mae']) / baseline_mae * 100
auc_actual = best_cls_row['test_roc_auc']

print(f"\n📊 Проверка бенчмарков (для horizon={HORIZON}):")
print(f"   MAE-gain: требуется ≥ {target_mae_gain}%, получено {mae_gain_actual:.2f}% {'✅' if mae_gain_actual >= target_mae_gain else '❌'}")
print(f"   AUC: требуется ≥ {target_auc:.2f}, получено {auc_actual:.4f} {'✅' if auc_actual >= target_auc else '❌'}")

print("\n" + "=" * 80)
print("✅ Baseline решение готово!")
print("=" * 80)


## Выводы

1. **Данные:** Успешно загружены и очищены данные OHLCV для BTC
2. **Признаки:** Построены технические индикаторы (returns, volatility, RSI, MACD, Bollinger Bands, volume)
3. **Целевые переменные:** 
   - Регрессия: forward return (процентное изменение через horizon баров)
   - Классификация: направление движения (вверх/вниз)
4. **Модели:** Реализованы 3 пары моделей:
   - Linear Regression + Logistic Regression (простые линейные модели)
   - Random Forest (ансамбли деревьев)
   - XGBoost/Gradient Boosting (с оптимизацией гиперпараметров)
5. **Метрики:** 
   - Регрессия: MAE, RMSE, R²
   - Классификация: Accuracy, Precision, Recall, F1-Score, ROC-AUC
6. **Продуктовые метрики:** MAE-gain, Sharpe ratio

### Следующие шаги:
- Добавить больше признаков (ончейн-данные, деривативы, макроэкономические индикаторы)
- Реализовать residual-таргет (отклонение от EMA/KAMA вместо простого forward return)
- Добавить калибровку вероятностей (Brier Score, ECE)
- Попробовать другие модели (LightGBM, CatBoost, нейронные сети)
- Улучшить feature engineering
- Реализовать walk-forward валидацию для более реалистичной оценки
- Добавить режимные гейты (разные модели для разных режимов рынка)
